In [1]:
# Import the libraries
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
from odc.stac import configure_rio, stac_load
import dask.distributed
import dask.utils
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# Set up Dask client for parallel processing
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)

# Configure rio with dynamic resolution
resolution = 20
memory_limit = dask.utils.parse_bytes(client.cluster.workers[0].memory_manager.memory_limit)
SHRINK = 4
if memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # Adjust chunk size if memory is limited

resolution = resolution * SHRINK


In [3]:
# Define the area of interest (AOI) for Lake Michigan
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-88.2, 43.0],  # Lower-left corner
            [-86.1, 43.0],  # Lower-right corner
            [-86.1, 45.0],  # Upper-right corner
            [-88.2, 45.0],  # Upper-left corner
            [-88.2, 43.0],  # Closing the polygon
        ]
    ],
}

#  time span of 10 years
time_of_interest = "2023-06-01/2023-07-01"

# Query the catalog for the data
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest
)
items = list(search.items())
print(f"Returned {len(items)} Items")


Returned 101 Items


In [4]:
# Load the data from the catalog with dynamic chunking and handle missing data
xx = stac_load(
    items,
    chunks={"x": 1024 * SHRINK, "y": 1024 * SHRINK},  # Dynamically adjust chunk size
    patch_url=planetary_computer.sign,
    resolution=resolution,
    dtype="uint16",  # Handle missing data by marking nodata values
    nodata=0
)

# Display loaded data
print(f"Bands: {','.join(list(xx.data_vars))}")
display(xx)


Bands: AOT,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,SCL,WVP,visual


<xarray.Dataset> Size: 9GB
Dimensions:      (y: 3874, x: 3873, time: 18)
Coordinates:
  * y            (y) float64 31kB 5e+06 5e+06 5e+06 ... 4.69e+06 4.69e+06
  * x            (x) float64 31kB 3e+05 3.001e+05 ... 6.097e+05 6.098e+05
    spatial_ref  int32 4B 32616
  * time         (time) datetime64[ns] 144B 2023-06-01T16:28:39.024000 ... 20...
Data variables: (12/16)
    AOT          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B01          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B02          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B03          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B04          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B05          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    ...           ...
    B11          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B12          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    B8A          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    SCL          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    WVP          (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>
    visual       (time, y, x) uint16 540MB dask.array<chunksize=(1, 3874, 3873), meta=np.ndarray>

In [5]:
# Function to convert data to float and handle missing nodata values
def to_float(xx, nodata_value=None):
    _xx = xx.astype("float32")  # Convert data to float32 for precision
    if nodata_value is None:
        nodata_value = _xx.attrs.pop("nodata", None)  # Fetch nodata value if exists
    if nodata_value is not None:
        return _xx.where(xx != nodata_value)  # Replace nodata with NaN
    return _xx

# Convert specific bands to float32 and handle missing data
b05 = to_float(xx.B05)  # Red-Edge band
b04 = to_float(xx.B04)  # Red band


In [6]:
# Calculate NDCI with small constant to avoid division by zero
ndci = (b05 - b04) / (b05 + b04 + 1e-6)

# Apply Min-Max normalization to scale NDCI between 0 and 1
ndci = (ndci - ndci.min()) / (ndci.max() - ndci.min())

# Display the calculated NDCI
display(ndci)


<xarray.DataArray (time: 18, y: 3874, x: 3873)> Size: 1GB
dask.array<truediv, shape=(18, 3874, 3873), dtype=float32, chunksize=(1, 3874, 3873), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 31kB 5e+06 5e+06 5e+06 ... 4.69e+06 4.69e+06
  * x            (x) float64 31kB 3e+05 3.001e+05 ... 6.097e+05 6.098e+05
    spatial_ref  int32 4B 32616
  * time         (time) datetime64[ns] 144B 2023-06-01T16:28:39.024000 ... 20...

In [7]:
# # Efficient concatenation of NDCI across the time dimension
# ndci_comp = xr.concat([ndci.isel(time=i) for i in range(len(ndci))], dim="time").compute()

# # Display concatenated time series
# print(ndci_comp)
# #WRONG

In [8]:
import dask.array as da

# Ensure ndci is a Dask array
ndci = ndci.chunk({"time": 1, "x": 1024, "y": 1024})

# Concatenate using Dask arrays
ndci_comp = da.concatenate([ndci.isel(time=i).data for i in range(ndci.sizes["time"])], axis=0)

# Reshape the concatenated array to include the time dimension
ndci_comp = ndci_comp.reshape((ndci.sizes["time"], ndci.sizes["y"], ndci.sizes["x"]))

# Convert back to xarray DataArray
ndci_comp = xr.DataArray(ndci_comp, dims=("time", "y", "x"), coords={"time": ndci.time, "y": ndci.y, "x": ndci.x})

# Display concatenated time series
print(ndci_comp)

<xarray.DataArray 'reshape-0db22df9515f80f5208b1991672800eb' (time: 18,
                                                              y: 3874, x: 3873)> Size: 1GB
dask.array<reshape, shape=(18, 3874, 3873), dtype=float32, chunksize=(1, 969, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 144B 2023-06-01T16:28:39.024000 ... 2023-0...
  * y        (y) float64 31kB 5e+06 5e+06 5e+06 ... 4.69e+06 4.69e+06 4.69e+06
  * x        (x) float64 31kB 3e+05 3.001e+05 3.002e+05 ... 6.097e+05 6.098e+05


In [9]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#save preprocessed xarray to a netCDF file
ndci_comp.to_netcdf("ndci_comp.nc")

In [2]:
import numpy as np
import xarray as xr
import dask.distributed

# Restart the Dask client
client = dask.distributed.Client()
client.restart()

# Load the saved NetCDF file
ndci_comp = xr.open_dataarray("ndci_comp.nc")

# Convert the Dask array to a NumPy array
ndci_comp_np = ndci_comp.compute()



2024-12-03 18:35:15,350 - distributed.nanny - WARNING - Restarting worker
2024-12-03 18:35:15,362 - distributed.nanny - WARNING - Restarting worker
2024-12-03 18:35:15,366 - distributed.nanny - WARNING - Restarting worker
2024-12-03 18:35:15,375 - distributed.nanny - WARNING - Restarting worker


In [ ]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 9.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()
ndci_values = ndci_comp_np.values.reshape(-1, 1)
ndci_scaled = scaler.fit_transform(ndci_values).reshape(ndci_comp_np.shape)

/Users/tishyakasliwal/anaconda3/envs/gdal_env/lib/python3.12/site-packages/sklearn/utils/_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/tishyakasliwal/anaconda3/envs/gdal_env/lib/python3.12/site-packages/sklearn/utils/_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [4]:
# Define a function to prepare the time series data for LSTM
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Set time steps (e.g., using last 10 values to predict the next)
time_steps = 10
X, y = create_dataset(ndci_scaled, time_steps)

# Reshape the input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], ndci_comp_np.shape[1] * ndci_comp_np.shape[2]))
y = np.reshape(y, (y.shape[0], ndci_comp_np.shape[1] * ndci_comp_np.shape[2]))

print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (8, 10, 15004002), y shape: (8, 15004002)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()

/Users/tishyakasliwal/anaconda3/envs/gdal_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 50)         │ 3,000,810,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15004002)       │   765,204,102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,766,034,902 (14.03 GB)

 Trainable params: 3,766,034,902 (14.03 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

# Plot the training and validation loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

# Evaluate the model
loss = model.evaluate(X, y)
print(f"Model loss: {loss}")

# Make predictions
predictions = model.predict(X)

# Reshape predictions to the original shape
predictions = predictions.reshape((predictions.shape[0], ndci_comp_np.shape[1], ndci_comp_np.shape[2]))

# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).reshape(predictions.shape)

# Plot some predictions
plt.figure(figsize=(10, 6))
plt.plot(ndci_comp.coords['time'].values[:len(predictions)], predictions[:, 0, 0], label='Predicted')
plt.plot(ndci_comp.coords['time'].values[:len(predictions)], ndci_comp_np[:len(predictions), 0, 0], label='Actual')
plt.legend()
plt.show()

Epoch 1/20


: 